In [144]:
import pandas as pd

Рассмотрим предоставленные нам данные

In [145]:
df_tr = pd.read_excel('customer_and_transaction.xlsx')

In [146]:
df_tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    20000 non-null  int64         
 1   product_id        20000 non-null  int64         
 2   customer_id       20000 non-null  int64         
 3   transaction_date  20000 non-null  datetime64[ns]
 4   online_order      19640 non-null  object        
 5   order_status      20000 non-null  object        
 6   brand             19803 non-null  object        
 7   product_line      19803 non-null  object        
 8   product_class     19803 non-null  object        
 9   product_size      19803 non-null  object        
 10  list_price        20000 non-null  float64       
 11  standard_cost     19803 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(6)
memory usage: 1.8+ MB


Датафрейм на 20 000 значений, 12 столбцов. Имеются пропуски в данных.

In [147]:
df_tr['transaction_id'].value_counts(sort=True, ascending=True).max()

1

ID транзакций уникальны, и могут быть использованны в качестве первичного ключа

In [148]:
df_c  = pd.read_excel('customer_and_transaction.xlsx', sheet_name=1)

In [149]:
df_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   customer_id            4000 non-null   int64 
 1   first_name             4000 non-null   object
 2   last_name              3875 non-null   object
 3   gender                 4000 non-null   object
 4   DOB                    3913 non-null   object
 5   job_title              3494 non-null   object
 6   job_industry_category  3344 non-null   object
 7   wealth_segment         4000 non-null   object
 8   deceased_indicator     4000 non-null   object
 9   owns_car               4000 non-null   object
 10  address                4000 non-null   object
 11  postcode               4000 non-null   int64 
 12  state                  4000 non-null   object
 13  country                4000 non-null   object
 14  property_valuation     4000 non-null   int64 
dtypes: int64(3), object(1

Датафрейм на 4 000 значений, 15 столбцов. Имеются пропуски в данных.

In [150]:
df_c['customer_id'].value_counts(sort=True, ascending=True).max()

1

ID клиентов уникальны, и могут быть использованны в качестве первичного ключа

In [151]:
df_c['gender'].value_counts()

gender
Female    2037
Male      1872
U           88
F            1
Femal        1
M            1
Name: count, dtype: int64

Пол клиентов записан разными способами, и вероятно лучше его привести к единному виду

In [152]:
df_c.loc[df_c['gender'].isin(['F', 'Femal']), 'gender'] = 'Female'

In [153]:
df_c.loc[df_c['gender'] == 'M', 'gender'] = 'Male'

In [154]:
df_c['gender'].value_counts()

gender
Female    2039
Male      1873
U           88
Name: count, dtype: int64

In [155]:
df_tr.tail(10)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
19990,19991,78,425,2017-01-26,False,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48
19991,19992,11,1374,2017-09-14,False,Approved,Giant Bicycles,Standard,high,medium,1274.93,764.96
19992,19993,13,5,2017-04-28,False,Approved,Solex,Standard,medium,medium,1163.89,589.27
19993,19994,77,2618,2017-12-23,False,Approved,Norco Bicycles,Road,medium,large,1240.31,795.10
19994,19995,9,718,2017-05-13,True,Approved,OHM Cycles,Road,medium,medium,742.54,667.40
19995,19996,51,1018,2017-06-24,True,Approved,OHM Cycles,Standard,high,medium,2005.66,1203.40
19996,19997,41,127,2017-11-09,True,Approved,Solex,Road,medium,medium,416.98,312.74
19997,19998,87,2284,2017-04-14,True,Approved,OHM Cycles,Standard,medium,medium,1636.90,44.71
19998,19999,6,2764,2017-07-03,False,Approved,OHM Cycles,Standard,high,medium,227.88,136.73
19999,20000,11,1144,2017-09-22,True,Approved,Trek Bicycles,Standard,medium,small,1775.81,1580.47


Один и тот же ID продукта может иметь разную продуктовую линейку, брэнд, размер и ценну. Соответственно для выделения его в отдельную таблицу понадобится сурогатный ключ

Выделим данные по продукту в отдельный датафрейм

In [156]:
df_pr= df_tr[['product_id', 'brand', 'product_line', 'product_class', 'product_size', 'list_price', 'standard_cost']]
df_pr.drop_duplicates().shape[0]

399

Датафрейм содержит 399 уникальных значений. Удалим дубликаты и проверим есть ли очевидная взаимосвязь между ценой и продуктом

In [157]:
df_pr = df_pr.drop_duplicates()
df_pr['product_id'].value_counts()

product_id
0     223
2       2
17      2
98      2
36      2
     ... 
55      1
3       1
47      1
94      1
65      1
Name: count, Length: 101, dtype: int64

In [158]:
df_pr[df_pr['product_id'] == 0].head(10)

,product_id,brand,product_line,product_class,product_size,list_price,standard_cost
34,0,Norco Bicycles,Road,medium,medium,544.05,376.84
54,0,Norco Bicycles,Standard,low,medium,363.01,290.41
60,0,OHM Cycles,Road,high,large,12.01,7.21
63,0,Trek Bicycles,Standard,medium,medium,499.53,388.72
82,0,OHM Cycles,Standard,medium,medium,235.63,125.07
99,0,Solex,Standard,medium,medium,478.16,298.72
110,0,OHM Cycles,Standard,low,medium,71.16,56.93
119,0,Trek Bicycles,Standard,high,medium,495.72,297.43
136,0,NaN,NaN,NaN,NaN,1942.61,NaN
159,0,NaN,NaN,NaN,NaN,1656.86,NaN


С учетом того, что у нас имеются продукты с ID 0 и полным отсутствием каких либо данных кроме цены, разумно вынести ценновые параметры в таблицу с транзакциями. Так как некоторое количество уникальных записей отличалось друг от друга только ценновыми параметрами, повторно удалим дубликаты, после удаления лишних столбцов

In [159]:
df_pr = df_pr.drop('list_price', axis=1)
df_pr = df_pr.drop('standard_cost', axis=1)
df_pr = df_pr.drop_duplicates()

Сгенерируем ключ для продуктов

In [160]:
df_pr = df_pr.reset_index().rename(columns={'index': 'id'})
df_pr.head()

,id,product_id,brand,product_line,product_class,product_size
0,0,2,Solex,Standard,medium,medium
1,1,3,Trek Bicycles,Standard,medium,large
2,2,37,OHM Cycles,Standard,low,medium
3,3,88,Norco Bicycles,Standard,medium,medium
4,4,78,Giant Bicycles,Standard,medium,large


In [161]:
df_pr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190 entries, 0 to 189
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             190 non-null    int64 
 1   product_id     190 non-null    int64 
 2   brand          189 non-null    object
 3   product_line   189 non-null    object
 4   product_class  189 non-null    object
 5   product_size   189 non-null    object
dtypes: int64(2), object(4)
memory usage: 9.0+ KB


Обновим ID продуктов в таблице транзакций и подготовим ее к выгрузке

In [162]:
df_tr = df_tr.merge(df_pr, on=['product_id', 'brand', 'product_line', 'product_class', 'product_size'], how='left')
df_tr = df_tr.drop(columns=['product_id', 'brand', 'product_line', 'product_class', 'product_size']).rename(columns={'id': 'product_id'})
df_tr.head()

,transaction_id,customer_id,transaction_date,online_order,order_status,list_price,standard_cost,product_id
0,1,2950,2017-02-25,False,Approved,71.49,53.62,0
1,2,3120,2017-05-21,True,Approved,2091.47,388.92,1
2,3,402,2017-10-16,False,Approved,1793.43,248.82,2
3,4,3135,2017-08-31,False,Approved,1198.46,381.10,3
4,5,787,2017-10-01,True,Approved,1765.30,709.48,4


Подготовим данны для загрузки в БД

In [163]:
df_tr.shape

(20000, 8)

In [168]:
df_tr.to_csv('transactions.csv', index=False)

In [165]:
df_pr.to_csv('products.csv', index=False)

In [166]:
df_c.to_csv('customers.csv', index=False)

При загрузке данных произошла ошибка, в связи с тем, что пользователь с ID 5034, не содержится в таблицу customers. Удалим соответвующую запись и посвторно создадим файл для загузки в БД

In [167]:
df_tr = df_tr[df_tr['customer_id'] != 5034]

In [169]:
df_tr.to_csv('transactions.csv', index=False)